In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
#pd.options.display.float_format = '{:.2f}'.format

# Load the data
data = pd.read_csv('../notebooks/data/merged_data.csv')

print(f'dataframe shape: {data.shape}')

dataframe shape: (1305, 344)


In [3]:

# create function to plot number of positive tests weekly for each drug class
def plot_drug_tests(df, type, drugclass, ax=None):
    """
    Plots the sum of drug test results for a specific drug class.

    Parameters:
    df (DataFrame): The input DataFrame containing the drug test data.
    type (str): The type of drug test, either 'test' or 'survey'.
    drugclass (str): The drug class to plot the results for.
    ax (Axes, optional): The matplotlib Axes object to plot the bar chart on. If not provided, a new Axes object will be created.

    Returns:
    ax (Axes): The matplotlib Axes object containing the bar chart.

    Raises:
    AssertionError: If the type is not 'test' or 'survey'.
    AssertionError: If the drugclass is not one of 'Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'.
    """

    # assert that type is either 'test' or 'survey'
    assert type in ['test', 'survey'], 'type must be either "test" or "survey"'

    # assert that drugclass must be include Propoxyphene, Amphetamines,Cannabinoids, Benzodiazepines, Cocaine (not case sensitive)
    assert drugclass in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'], 'drugclass must be one of "Propoxyphene", "Amphetamines", "Methamphetamine", "Cannabinoids", "Benzodiazepines", "Cocaine" or "Opiate300"'

    # filter df to total_visits == 26, this filters patients that completed treatment
    df = df[df['total_visits'] == 26]

    # create dataframe with columns for type and drug class
    df = df[[col for col in df.columns if col.startswith(type+'_') and drugclass in col]]

    # remove text and leave numbers in column names
    df.columns = [col.split('_')[-1] for col in df.columns]

    # fill nan values with 0
    df = df.fillna(0)

    # replace -5.0 with 0.0 
    df = df.replace(-5.0, 0.0)

    # plot the sum of the columns

    # set condition if axis object is not passed
    if ax is None:
        ax = plt.gca()
    sns.barplot(x=df.columns, y=df.sum(), ax=ax, palette='Blues_d')
    
    # create title
    ax.set_title(f' {drugclass} tests', fontsize=15)
    
    # remove the x axis 
    ax.set_xticklabels('')

    return ax

In [4]:

# create function to plot number of positive tests weekly for each drug class
def drug_test_df(df, type, drugclass, ax=None):
    """
    Plots the sum of drug test results for a specific drug class.

    Parameters:
    df (DataFrame): The input DataFrame containing the drug test data.
    type (str): The type of drug test, either 'test' or 'survey'.
    drugclass (str): The drug class to plot the results for.
    ax (Axes, optional): The matplotlib Axes object to plot the bar chart on. If not provided, a new Axes object will be created.

    Returns:
    ax (Axes): The matplotlib Axes object containing the bar chart.

    Raises:
    AssertionError: If the type is not 'test' or 'survey'.
    AssertionError: If the drugclass is not one of 'Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'.
    """

    # assert that type is either 'test' or 'survey'
    assert type in ['test', 'survey'], 'type must be either "test" or "survey"'

    # assert that drugclass must be include Propoxyphene, Amphetamines,Cannabinoids, Benzodiazepines, Cocaine (not case sensitive)
    assert drugclass in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine', 'cannabis','oxycodone','methadone','amphetamine','methamphetamine','opiates','benzodiazepines','propoxyphene'], 'drugclass must be one of "Propoxyphene", "Amphetamines", "Methamphetamine", "Cannabinoids", "Benzodiazepines", "Cocaine" "Opiate300" or "opiates"'

    # filter df to total_visits == 26, this filters patients that completed treatment
    df = df[df['total_visits'] == 26]

    # create dataframe with columns for type and drug class
    df = df[[col for col in df.columns if col.startswith(type+'_') and drugclass in col]]

    # remove text and leave numbers in column names
    df.columns = [col.split('_')[-1] for col in df.columns]

    # fill nan values with 0
    df = df.fillna(0)

    # replace -5.0 with 0.0 
    df = df.replace(-5.0, 0.0)

    df = df.sum()

    df = pd.DataFrame(df)

    df = df.rename(columns={0: f'{drugclass[:3].upper()}'})

    return df

In [5]:
data1 = data.loc[(data.responder==1)&(data.total_visits==26)]
data0 = data.loc[(data.responder==0)&(data.total_visits==26)]

In [6]:
# create dataframe with number of drug test per drug class by treatment outcome

# these are the patients that responded to treatment

# create for loop to create a df for each drug class
for drug in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']:
 # turn each drug class into a dataframe
 globals()[f'{drug}'] = drug_test_df(data1, 'test', drug)

# merge all the dataframes
drug_test_r1 = pd.concat([Opiate300, Propoxyphene, Amphetamines, Methamphetamine, Cannabinoids, Benzodiazepines, Cocaine], axis=1)

# add '-r1' to the end of each column name
drug_test_r1.columns = [f'{col}_r1' for col in drug_test_r1.columns]

drug_test_r1

,OPI_r1,PRO_r1,AMP_r1,MET_r1,CAN_r1,BEN_r1,COC_r1
0,212.0,8.0,11.0,14.0,57.0,49.0,71.0
1,81.0,2.0,12.0,16.0,62.0,31.0,60.0
2,66.0,1.0,12.0,15.0,64.0,21.0,63.0
3,58.0,3.0,10.0,14.0,56.0,18.0,62.0
4,47.0,1.0,10.0,10.0,60.0,18.0,60.0
5,44.0,2.0,14.0,14.0,55.0,17.0,49.0
6,40.0,0.0,14.0,14.0,54.0,30.0,51.0
7,35.0,2.0,10.0,10.0,56.0,29.0,59.0
8,32.0,4.0,9.0,12.0,51.0,24.0,60.0
9,35.0,4.0,11.0,15.0,57.0,24.0,55.0


In [7]:
# create dataframe with number of drug test per drug class by treatment outcome

# these are the patients that did not respond to treatment

# create for loop to create a df for each drug class
for drug in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']:
 # turn each drug class into a dataframe
 globals()[f'{drug}'] = drug_test_df(data0, 'test', drug)

# merge all the dataframes
drug_test_r0 = pd.concat([Opiate300, Propoxyphene, Amphetamines, Methamphetamine, Cannabinoids, Benzodiazepines, Cocaine], axis=1)

# add '_r1' to the end of each column name
drug_test_r0.columns = [f'{col}_r0' for col in drug_test_r0.columns]

drug_test_r0


,OPI_r0,PRO_r0,AMP_r0,MET_r0,CAN_r0,BEN_r0,COC_r0
0,423.0,6.0,26.0,35.0,108.0,90.0,187.0
1,249.0,6.0,21.0,29.0,100.0,80.0,159.0
2,224.0,5.0,23.0,31.0,94.0,63.0,167.0
3,199.0,3.0,27.0,31.0,91.0,59.0,155.0
4,206.0,4.0,22.0,29.0,105.0,58.0,157.0
5,182.0,2.0,24.0,33.0,100.0,62.0,136.0
6,229.0,3.0,19.0,25.0,100.0,86.0,148.0
7,205.0,4.0,25.0,29.0,98.0,70.0,157.0
8,208.0,4.0,28.0,33.0,110.0,70.0,141.0
9,189.0,5.0,22.0,28.0,97.0,67.0,142.0


In [8]:
def calculate_test_pvalues(group1, group2):
    
    tests = []
    ci = []
    cd_list = []  
    bf10 = []
    power = []

    # perform ttest between each drug class
    for drug in ['OPI','PRO','AMP','MET','CAN','BEN','COC']:
        t = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['p-val'][0]
        tests.append(t)
        c = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['CI95%'][0].tolist()
        c = ', '.join(map(str, c))
        ci.append(c)
        cd = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['cohen-d'][0]
        cd_list.append(cd)  
        b = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['BF10'][0]
        bf10.append(b)
        p = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['power'][0]
        power.append(p)

    test_pvs = pd.DataFrame(
        {'drugClass':['OPI','PRO','AMP','MET','CAN','BEN','COC'],
         'pvalue':tests,
         'CI95%':ci,
         'cohen-d':cd_list,
         'BF10:':bf10,
         'power':power})

    return test_pvs

test_pvs = calculate_test_pvalues(drug_test_r1, drug_test_r0)

test_pvs

,drugClass,pvalue,CI95%,cohen-d,BF10:,power
0,OPI,1.464256e-18,"-204.4, -153.04",3.957760,2.0e+15,1.000000
1,PRO,4.031098e-03,"-2.27, -0.45",0.854462,9.944,0.841359
2,AMP,2.614400e-22,"-19.55, -15.49",4.905620,8.486e+18,1.000000
3,MET,1.094534e-25,"-25.4, -20.92",5.891839,1.624e+22,1.000000
4,CAN,6.677985e-28,"-46.43, -39.09",6.619719,2.325e+24,1.000000
5,BEN,2.746061e-25,"-48.65, -39.91",5.767780,6.638e+21,1.000000
6,COC,7.019927e-35,"-103.27, -91.53",9.443339,1.493e+31,1.000000


In [9]:
# create loop to create a df for each drug class for survey data

for drug in ['cannabis','oxycodone','methadone','amphetamine','methamphetamine','opiates','benzodiazepines','propoxyphene']:
    # turn each drug class into a dataframe
    globals()[f'{drug}'] = drug_test_df(data1, 'survey', drug)
    
# merge all the dataframes
from functools import reduce

drug_survey_r1 = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), [cannabis, oxycodone, methadone, amphetamine, methamphetamine, opiates, benzodiazepines, propoxyphene])

# add '-r1' to the end of each column name
drug_survey_r1.columns = [f'{col}_r1' for col in drug_survey_r1.columns]

drug_survey_r1

,CAN_r1,OXY_r1,MET_x_r1,AMP_r1,MET_y_r1,OPI_r1,BEN_r1,PRO_r1
0,796.0,1145.0,543.0,52.0,76.0,6151.0,194.0,4.0
0,796.0,1145.0,543.0,76.0,76.0,6151.0,194.0,4.0
12,695.0,20.0,1.0,30.0,94.0,209.0,83.0,6.0
12,695.0,20.0,1.0,94.0,94.0,209.0,83.0,6.0
16,655.0,37.0,4.0,18.0,49.0,63.0,85.0,5.0
16,655.0,37.0,4.0,49.0,49.0,63.0,85.0,5.0
20,738.0,8.0,8.0,6.0,45.0,24.0,94.0,4.0
20,738.0,8.0,8.0,45.0,45.0,24.0,94.0,4.0
24,688.0,10.0,12.0,15.0,58.0,18.0,52.0,3.0
24,688.0,10.0,12.0,58.0,58.0,18.0,52.0,3.0


In [10]:
# change MET_x_r1 to MET_r1 and MET_y_r1 to MAM_r1
drug_survey_r1.columns = drug_survey_r1.columns.str.replace('MET_x_r1', 'MET_r1')
drug_survey_r1.columns = drug_survey_r1.columns.str.replace('MET_y_r1', 'MAM_r1')

drug_survey_r1

,CAN_r1,OXY_r1,MET_r1,AMP_r1,MAM_r1,OPI_r1,BEN_r1,PRO_r1
0,796.0,1145.0,543.0,52.0,76.0,6151.0,194.0,4.0
0,796.0,1145.0,543.0,76.0,76.0,6151.0,194.0,4.0
12,695.0,20.0,1.0,30.0,94.0,209.0,83.0,6.0
12,695.0,20.0,1.0,94.0,94.0,209.0,83.0,6.0
16,655.0,37.0,4.0,18.0,49.0,63.0,85.0,5.0
16,655.0,37.0,4.0,49.0,49.0,63.0,85.0,5.0
20,738.0,8.0,8.0,6.0,45.0,24.0,94.0,4.0
20,738.0,8.0,8.0,45.0,45.0,24.0,94.0,4.0
24,688.0,10.0,12.0,15.0,58.0,18.0,52.0,3.0
24,688.0,10.0,12.0,58.0,58.0,18.0,52.0,3.0


In [11]:
# create loop to create a df for each drug class for survey data

for drug in ['cannabis','oxycodone','methadone','amphetamine','methamphetamine','opiates','benzodiazepines','propoxyphene']:
    # turn each drug class into a dataframe
    globals()[f'{drug}'] = drug_test_df(data0, 'survey', drug)
    
# merge all the dataframes
from functools import reduce

drug_survey_r0 = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), [cannabis, oxycodone, methadone, amphetamine, methamphetamine, opiates, benzodiazepines, propoxyphene])

# add '-r1' to the end of each column name
drug_survey_r0.columns = [f'{col}_r0' for col in drug_survey_r0.columns]

# rename columns
drug_survey_r0

drug_survey_r0

,CAN_r0,OXY_r0,MET_x_r0,AMP_r0,MET_y_r0,OPI_r0,BEN_r0,PRO_r0
0,986.0,1026.0,618.0,71.0,170.0,11815.0,233.0,3.0
0,986.0,1026.0,618.0,170.0,170.0,11815.0,233.0,3.0
12,1099.0,85.0,5.0,11.0,259.0,1505.0,221.0,0.0
12,1099.0,85.0,5.0,259.0,259.0,1505.0,221.0,0.0
16,1228.0,95.0,13.0,10.0,267.0,1219.0,197.0,1.0
16,1228.0,95.0,13.0,267.0,267.0,1219.0,197.0,1.0
20,1167.0,47.0,13.0,9.0,264.0,1200.0,221.0,1.0
20,1167.0,47.0,13.0,264.0,264.0,1200.0,221.0,1.0
24,1050.0,72.0,18.0,55.0,255.0,1600.0,265.0,1.0
24,1050.0,72.0,18.0,255.0,255.0,1600.0,265.0,1.0


In [12]:
# change MET_x_r0 to MET_r0 and MET_y_r0 to MAM_r0
drug_survey_r0.columns = drug_survey_r0.columns.str.replace('MET_x_r0', 'MET_r0')
drug_survey_r0.columns = drug_survey_r0.columns.str.replace('MET_y_r0', 'MAM_r0')

drug_survey_r0

,CAN_r0,OXY_r0,MET_r0,AMP_r0,MAM_r0,OPI_r0,BEN_r0,PRO_r0
0,986.0,1026.0,618.0,71.0,170.0,11815.0,233.0,3.0
0,986.0,1026.0,618.0,170.0,170.0,11815.0,233.0,3.0
12,1099.0,85.0,5.0,11.0,259.0,1505.0,221.0,0.0
12,1099.0,85.0,5.0,259.0,259.0,1505.0,221.0,0.0
16,1228.0,95.0,13.0,10.0,267.0,1219.0,197.0,1.0
16,1228.0,95.0,13.0,267.0,267.0,1219.0,197.0,1.0
20,1167.0,47.0,13.0,9.0,264.0,1200.0,221.0,1.0
20,1167.0,47.0,13.0,264.0,264.0,1200.0,221.0,1.0
24,1050.0,72.0,18.0,55.0,255.0,1600.0,265.0,1.0
24,1050.0,72.0,18.0,255.0,255.0,1600.0,265.0,1.0


In [13]:
def calculate_survey_pvalues(group1, group2):
    
    tests = []
    ci = []
    cd_list = []  
    bf10 = []
    power = []

    # perform ttest between each drug class
    for drug in ['CAN','OXY','MET','AMP','MAM','OPI','BEN','PRO']:
        t = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['p-val'][0]
        tests.append(t)
        c = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['CI95%'][0].tolist()
        c = ', '.join(map(str, c))
        ci.append(c)
        cd = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['cohen-d'][0]
        cd_list.append(cd)  
        b = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['BF10'][0]
        bf10.append(b)
        p = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['power'][0]
        power.append(p)

    survey_pvs = pd.DataFrame(
        {'drugClass':['CAN','OXY','MET','AMP','MAM','OPI','BEN','PRO'],
         'pvalue':tests,
         'CI95%':ci,
         'cohen-d':cd_list,
         'BF10:':bf10,
         'power':power})

    return survey_pvs

survey_pvs = calculate_survey_pvalues(drug_survey_r1, drug_survey_r0)

survey_pvs

,drugClass,pvalue,CI95%,cohen-d,BF10:,power
0,CAN,1.428140e-14,"-437.74, -334.54",5.813936,2.368e+11,1.000000
1,OXY,8.592200e-01,"-315.43, 264.86",0.067707,0.357,0.053420
2,MET,8.377031e-01,"-176.52, 144.24",0.078199,0.359,0.054566
3,AMP,1.029607e-02,"-145.54, -21.46",1.045582,5.0,0.758975
4,MAM,2.268241e-13,"-185.89, -137.25",5.161557,1.721e+10,1.000000
5,OPI,8.910096e-02,"-4418.18, 334.47",0.667566,1.099,0.397954
6,BEN,6.998432e-10,"-162.32, -104.25",3.566552,9.065e+06,1.000000
7,PRO,1.947329e-04,"1.28, 3.58",1.638161,118.46,0.986420
